# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['Population (2016 est.)[8]']= cities['Population (2016 est.)[8]'].astype(float)

def get_ciudad(x):
    if x['palabras']==3:
        return x['team'].split(" ")[0]+" "+x['team'].split(" ")[1]
    else:
        return x['team'].split(" ")[0]

def nhl_correlation(): 
    nhl = nhl_df[nhl_df.year==2018]
    nhl = nhl.drop([0,9,18,26])


    nhl['palabras'] = [len(team.split(" ")) for team in nhl['team'] ]
    nhl['Metropolitan area'] = nhl.apply(get_ciudad, axis=1)

    nhl['Metropolitan area'].replace({
        'Columbus Blue' : 'Columbus',
        'Colorado' : 'Denver',
        'Dallas':'Dallas–Fort Worth',
        'Detroit Red':'Detroit',
        'Florida': 'Miami–Fort Lauderdale',
        'Vegas Golden':'Las Vegas',
        'New York': 'New York City',
        'New Jersey' : 'New York City',
        'Minnesota':'Minneapolis–Saint Paul',
        'Tampa Bay': 'Tampa Bay Area',
        'Toronto Maple': 'Toronto',
        'Washington': 'Washington, D.C.',
        'Arizona': 'Phoenix',
        'Carolina':'Raleigh',
        'San Jose': 'San Francisco Bay Area',
        'Anaheim': 'Los Angeles'
    }, inplace=True)

    nhl.sort_values(by=['Metropolitan area'])
    nhl['W'] = nhl['W'].astype(float)
    nhl['L'] = nhl['L'].astype(float)

    q1 = nhl.groupby(['Metropolitan area']).mean()

    q1 = pd.merge(cities,q1,how="inner" ,on='Metropolitan area').sort_values(by=['Metropolitan area'])
    q1['W/L'] = q1['W'].astype(float)/(q1['W'].astype(float)+q1['L'].astype(float))
    q1[['Metropolitan area','W/L','Population (2016 est.)[8]']]
   
    population_by_region = q1['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = q1['W/L'].tolist()
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [41]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    nba= nba_df[nba_df.year==2018]
    nba = nba.replace('\(\d+\)',"",regex=True)
    nba = nba.replace('\*',"",regex=True)
    nba = nba.replace('\\xa0',"",regex=True)
    nba['NBA'] = [team.split(" ")[-1] for team in nba['team']]

    nba['NBA'].replace({
        'Clippers' : 'LakersClippers',
        'Lakers': 'LakersClippers',
        'Knicks':'KnicksNets',
        'Nets':'KnicksNets',
        'Blazers' : 'Trail Blazers'

    }, inplace=True)
    nba['W'] = nba['W'].astype(float)
    nba['L'] = nba['L'].astype(float)
    nba=nba.groupby("NBA").mean()

    ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    ciudades=ciudades.replace('\[.*\]',"",regex=True)
    ciudades=ciudades.replace(['—',''],np.nan).dropna()

    q2 = pd.merge(ciudades,nba ,how="inner",on='NBA').sort_values('Metropolitan area')
    q2['W/L'] = q2['W'].astype(float)/(q2['W'].astype(float)+q2['L'].astype(float))
    q2['Population (2016 est.)[8]'] = q2['Population (2016 est.)[8]'].astype(float)
    q2[['Metropolitan area','W/L','Population (2016 est.)[8]']]
    
    population_by_region = q2['Population (2016 est.)[8]'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = q2['W/L'].tolist()# pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [42]:
nba_correlation()

-0.17657160252844617

In [32]:
nba= nba_df[nba_df.year==2018]
nba = nba.replace('\(\d+\)',"",regex=True)
nba = nba.replace('\*',"",regex=True)
nba = nba.replace('\\xa0',"",regex=True)
nba['NBA'] = [team.split(" ")[-1] for team in nba['team']]
#str1 = " " 
#nba['ciudad'] = [str1.join(team.split(" ")[0:-1]) for team in nba['team']]
nba['NBA'].replace({
    'Clippers' : 'LakersClippers',
    'Lakers': 'LakersClippers',
    'Knicks':'KnicksNets',
    'Nets':'KnicksNets',
    'Blazers' : 'Trail Blazers'

}, inplace=True)
nba['W'] = nba['W'].astype(float)
nba['L'] = nba['L'].astype(float)
nba=nba.groupby("NBA").mean()

ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
ciudades=ciudades.replace('\[.*\]',"",regex=True)
ciudades=ciudades.replace(['—',''],np.nan).dropna()

q2 = pd.merge(ciudades,nba ,how="inner",on='NBA').sort_values('Metropolitan area')
q2['W/L'] = q2['W'].astype(float)/(q2['W'].astype(float)+q2['L'].astype(float))
q2[['Metropolitan area','W/L','Population (2016 est.)[8]']]


,Metropolitan area,W/L,Population (2016 est.)[8]
15,Atlanta,0.292683,5789700
7,Boston,0.670732,4794447
17,Charlotte,0.439024,2474314
3,Chicago,0.329268,9512999
16,Cleveland,0.609756,2055612
4,Dallas–Fort Worth,0.292683,7233323
9,Denver,0.560976,2853077
12,Detroit,0.475610,4297617
14,Houston,0.792683,6772470
18,Indianapolis,0.585366,2004230


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [59]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb= mlb_df[mlb_df.year==2018]
    mlb['team'].replace({
        'Boston Red Sox' : 'Boston RedSox',
        'Toronto Blue Jays': 'Toronto BlueJays'

    }, inplace=True)

    mlb['MLB'] = [team.split(" ")[-1] for team in mlb['team']]

    mlb['MLB'].replace({
        'Sox' : 'CubsWhite Sox',
        'Cubs' : 'CubsWhite Sox',
        'RedSox': 'Red Sox',
        'Dodgers': 'DodgersAngels',
        'Angels' : 'DodgersAngels',
        'Yankees': 'YankeesMets',
        'Mets': 'YankeesMets',
        'Athletics': 'GiantsAthletics',
        'Giants' :'GiantsAthletics',
        'BlueJays' : 'Blue Jays'

    }, inplace=True)

    mlb['W'] = mlb['W'].astype(float)
    mlb['L'] = mlb['L'].astype(float)
    mlb=mlb.groupby("MLB").mean()
    
    ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    ciudades=ciudades.replace('\[.*\]',"",regex=True)
    ciudades=ciudades.replace(['—',''],np.nan).dropna()

    q3 = pd.merge(ciudades,mlb ,how="inner",on='MLB').sort_values('Metropolitan area')
    q3['W/L'] = q3['W']/(q3['W']+q3['L'])
    q3['Population (2016 est.)[8]'] = q3['Population (2016 est.)[8]'].astype(float)
    q3[['Metropolitan area','W/L','Population (2016 est.)[8]']]
    
    population_by_region = q3['Population (2016 est.)[8]'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = q3['W/L'].tolist() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [60]:
mlb_correlation()

0.1505230448710485

In [55]:
mlb= mlb_df[mlb_df.year==2018]
mlb['team'].replace({
    'Boston Red Sox' : 'Boston RedSox',
    'Toronto Blue Jays': 'Toronto BlueJays'

}, inplace=True)

mlb['MLB'] = [team.split(" ")[-1] for team in mlb['team']]

mlb['MLB'].replace({
    'Sox' : 'CubsWhite Sox',
    'Cubs' : 'CubsWhite Sox',
    'RedSox': 'Red Sox',
    'Dodgers': 'DodgersAngels',
    'Angels' : 'DodgersAngels',
    'Yankees': 'YankeesMets',
    'Mets': 'YankeesMets',
    'Athletics': 'GiantsAthletics',
    'Giants' :'GiantsAthletics',
    'BlueJays' : 'Blue Jays'
    
    


}, inplace=True)

mlb['W'] = mlb['W'].astype(float)
mlb['L'] = mlb['L'].astype(float)
mlb=mlb.groupby("MLB").mean()

ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
ciudades=ciudades.replace('\[.*\]',"",regex=True)
ciudades=ciudades.replace(['—',''],np.nan).dropna()

q3 = pd.merge(ciudades,mlb ,how="inner",on='MLB').sort_values('Metropolitan area')
q3['W/L'] = q3['W']/(q3['W']+q3['L'])
q3[['Metropolitan area','W/L','Population (2016 est.)[8]']]


,W,L,W-L%,year
MLB,,,,
Astros,103.0,59.0,0.636,2018
Blue Jays,73.0,89.0,0.451,2018
Braves,90.0,72.0,0.556,2018
Brewers,96.0,67.0,0.589,2018
Cardinals,88.0,74.0,0.543,2018
CubsWhite Sox,78.5,84.0,0.483,2018
Diamondbacks,82.0,80.0,0.506,2018
DodgersAngels,86.0,76.5,0.529,2018
GiantsAthletics,85.0,77.0,0.525,2018


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [104]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl= nfl_df[nfl_df.year==2018]
    nfl = nfl.drop([5*i for i in range(8)])

    nfl['NFL'] = [team.split(" ")[-1] for team in nfl['team']]
    nfl['NFL'] = nfl['NFL'].replace("\W","",regex=True)

    nfl['NFL'].replace({
        'Giants' : 'GiantsJets',
        'Jets' : 'GiantsJets',
        'Rams' : 'RamsChargers',
        'Chargers' : 'RamsChargers',
        '49ers' : '49ersRaiders',
        'Raiders' : '49ersRaiders',
    }, inplace=True)


    nfl['W'] = nfl['W'].astype(float)
    nfl['L'] = nfl['L'].astype(float)
    nfl=nfl.groupby("NFL").mean()
    
    ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    ciudades=ciudades.replace('\[.*\]',"",regex=True)
    ciudades=ciudades.replace(['—',''],np.nan).dropna()

    q4 = pd.merge(ciudades,nfl ,how="inner",on='NFL')
    q4['W/L'] = q4['W']/(q4['W']+q4['L'])
    q4['Population (2016 est.)[8]'] = q4['Population (2016 est.)[8]'].astype(float)
    q4[['Metropolitan area','W/L','Population (2016 est.)[8]']]
    
    population_by_region = q4['Population (2016 est.)[8]'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = q4['W/L'].tolist() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [105]:
nfl_correlation()

0.004922112149349386

In [103]:
nfl= nfl_df[nfl_df.year==2018]
nfl = nfl.drop([5*i for i in range(8)])
"""
nfl['team'].replace({
    'Boston Red Sox' : 'Boston RedSox',
    'Toronto Blue Jays': 'Toronto BlueJays'

}, inplace=True)

"""
nfl['NFL'] = [team.split(" ")[-1] for team in nfl['team']]
nfl['NFL'] = nfl['NFL'].replace("\W","",regex=True)

nfl['NFL'].replace({
    'Giants' : 'GiantsJets',
    'Jets' : 'GiantsJets',
    'Rams' : 'RamsChargers',
    'Chargers' : 'RamsChargers',
    '49ers' : '49ersRaiders',
    'Raiders' : '49ersRaiders',
}, inplace=True)


nfl['W'] = nfl['W'].astype(float)
nfl['L'] = nfl['L'].astype(float)
nfl=nfl.groupby("NFL").mean()
nfl

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year,NFL
1,2.1,5.0,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11.0,.688,New England Patriots*,2018,Patriots
2,-5.2,9.0,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7.0,.438,Miami Dolphins,2018,Dolphins
3,-0.6,10.0,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6.0,.375,Buffalo Bills,2018,Bills
4,-5.9,12.0,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4.0,.250,New York Jets,2018,GiantsJets
6,6.4,6.0,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10.0,.625,Baltimore Ravens*,2018,Ravens
7,1.7,6.0,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9.0,.594,Pittsburgh Steelers,2018,Steelers
8,0.6,8.0,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7.0,.469,Cleveland Browns,2018,Browns
9,-3.4,10.0,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6.0,.375,Cincinnati Bengals,2018,Bengals
11,1.4,5.0,NFL,5.4,2.4,316,86,402,3.8,-1.5,0,11.0,.688,Houston Texans*,2018,Texans
12,-0.6,6.0,NFL,5.6,3.9,344,89,433,3.4,-2.2,0,10.0,.625,Indianapolis Colts+,2018,Colts


In [101]:
ciudades = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
ciudades=ciudades.replace('\[.*\]',"",regex=True)
ciudades=ciudades.replace(['—',''],np.nan).dropna()
ciudades

,Metropolitan area,Population (2016 est.)[8],NFL
0,New York City,20153634,GiantsJets
1,Los Angeles,13310447,RamsChargers
2,San Francisco Bay Area,6657982,49ersRaiders
3,Chicago,9512999,Bears
4,Dallas–Fort Worth,7233323,Cowboys
5,"Washington, D.C.",6131977,Redskins
6,Philadelphia,6070500,Eagles
7,Boston,4794447,Patriots
8,Minneapolis–Saint Paul,3551036,Vikings
9,Denver,2853077,Broncos


In [102]:
q4 = pd.merge(ciudades,nfl ,how="inner",on='NFL')
q4['W/L'] = q4['W']/(q4['W']+q4['L'])
q4[['Metropolitan area','W/L','Population (2016 est.)[8]']]

,Metropolitan area,W/L,Population (2016 est.)[8]
0,New York City,0.281250,20153634
1,Los Angeles,0.781250,13310447
2,San Francisco Bay Area,0.250000,6657982
3,Chicago,0.750000,9512999
4,Dallas–Fort Worth,0.625000,7233323
5,"Washington, D.C.",0.437500,6131977
6,Philadelphia,0.562500,6070500
7,Boston,0.687500,4794447
8,Minneapolis–Saint Paul,0.533333,3551036
9,Denver,0.375000,2853077


In [80]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [106]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values